In [1]:
#Realizado por Luis Burbano, Cesar Loor, Sebastian Torres

In [2]:
import pandas as pd
from sentence_transformers import SentenceTransformer
import chromadb
from chromadb.config import Settings
from chromadb import Client
from langchain_community.llms.ollama import Ollama
from langchain_core.messages import HumanMessage, AIMessage
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

C:\Users\Liak\AppData\Roaming\Python\Python311\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


In [3]:
# Cargar el archivo CSV con delimitador ;
file_path = 'Base_conocimiento_pre.csv'  # Actualiza el path si es necesario
df = pd.read_csv(file_path, delimiter=';')

In [4]:
# Limpiar datos faltantes en la columna 'Pregunta'
df['Pregunta'] = df['Pregunta'].fillna('')

In [5]:
# Cargar el modelo de embeddings
model = SentenceTransformer('all-MiniLM-L6-v2')

In [6]:
# Crear embeddings para las preguntas
df['question_embeddings'] = df['Pregunta'].apply(lambda x: model.encode(x).tolist())

In [7]:
# Configurar Chroma
settings = Settings()
client = Client(settings=settings)

In [8]:
# Crear una colección
collection = client.create_collection(name="qa_collection")

In [9]:
# Agregar vectores a la colección
for index, row in df.iterrows():
    collection.add(
        documents=[row['Pregunta']],
        embeddings=[row['question_embeddings']],
        ids=[str(index)]  # Cambiado metadatas a ids
    )

In [10]:
def obtener_respuesta(pregunta: str, collection, df, model):
    """
    Obtiene la mejor respuesta para una pregunta dada usando la colección y el modelo de embeddings.

    Args:
    - pregunta (str): La pregunta para la cual obtener una respuesta.
    - collection: La colección de documentos en Chroma.
    - df: El DataFrame que contiene las preguntas y respuestas.
    - model: El modelo de embeddings para generar la representación de la pregunta.

    Returns:
    - Tuple[str, float]: La mejor respuesta correspondiente a la pregunta y la distancia de similitud.
    """
    # Codificar la pregunta
    query_embedding = model.encode(pregunta).tolist()

    # Realizar la consulta en la colección
    results = collection.query(query_embeddings=[query_embedding], n_results=1)

    # Obtener las listas de IDs y distancias
    ids_list = results['ids'][0]
    distances_list = results['distances'][0]

    if not ids_list:  # No hay resultados
        return None, float('inf')

    # Combina los IDs y distancias en un solo objeto
    results_combined = list(zip(ids_list, distances_list))

    # Ordena los resultados por distancia (menor distancia significa mayor similitud)
    sorted_results = sorted(results_combined, key=lambda x: x[1])

    # Selecciona solo el mejor resultado (menor distancia)
    best_result = sorted_results[0]  # Solo el primer resultado es el mejor

    doc_id, distance = best_result
    index = int(doc_id)

    # Obtener la respuesta desde el DataFrame
    respuesta = df.loc[index, 'Respuesta']

    return respuesta, distance

In [11]:
# Configuración del modelo Llama3
llm = Ollama(model="gemma2")


In [12]:
prompt_template = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            """Eres una IA llamada Espesito. Tu tarea principal es responder preguntas simples sobre la Universidad de las Fuerzas Armadas "ESPE", basándote en la información proporcionada en las preguntas frecuentes.
            Cuando no tengas una respuesta exacta, responde de acuerdo a lo que has aprendido y el contexto dado en el historial de chat.""",
        ),
        MessagesPlaceholder(variable_name="chat_history"),
        ("human", "{input}"),
    ]
)


In [13]:
chain = prompt_template | llm

In [14]:
def chat():
    chat_history = []
    umbral_similitud = 0.5  # Ajusta este umbral según sea necesario
    while True:
        pregunta = input("You: ")
        if pregunta.lower() == "adios":
            return
        
        # Primero, intenta obtener una respuesta de Chroma
        answer, distance = obtener_respuesta(pregunta, collection, df, model)
        
        if answer and distance < umbral_similitud:
            response = answer
        else:
            # Si no hay respuesta en Chroma o la similitud es baja, utiliza el modelo Llama3
            response = chain.invoke({"input": pregunta, "chat_history": chat_history})
        
        chat_history.append(HumanMessage(content=pregunta))
        chat_history.append(AIMessage(content=response))
        print("_" * 50)
        print("IA: " + response)

In [15]:
chat()

You:  Hola!


__________________________________________________
IA: ¡Hola! 👋 ¿En qué puedo ayudarte hoy sobre la Universidad de las Fuerzas Armadas ESPE? 😊 



You:  Quiero estudiar una carrera enfocada al desarrollo web, ¿cual me recomiendas?


__________________________________________________
IA: ¡Excelente pregunta! Para el desarrollo web en ESPE, te recomendaría que revises la carrera de **Ingeniería en Sistemas Informáticos**.  Esta carrera te proporciona las bases en programación, diseño web y estructuras de datos que necesitas para un trabajo exitoso en desarrollo web. 

¿Te gustaría saber más sobre esta carrera o hay algo específico del desarrollo web que te interese? 😊



You:  ¿La ESPE es una universidad gratuita?


...

You:  ¿Quien es coordinador de pasantías del DCCO?


__________________________________________________
IA: Phd. Edison Jorge Lascano 


You:  Si necesito hacer pasantías y soy del DCCO, ¿con quién debo hablar?


__________________________________________________
IA: Para hacer pasantías en el DCCO, deberías comunicarte con el **Phd. Edison Jorge Lascano**,  el coordinador de pasantías. 😊

¿Hay algo más que te gustaría saber sobre las pasantías? 





You:  adios


In [6]:
import chromadb
import numpy as np
import pandas as pd
from sentence_transformers import SentenceTransformer, util
from langchain.embeddings import SentenceTransformerEmbeddings
from langchain.vectorstores import Chroma
from langchain.llms import Ollama

class ChatbotModel:
    def __init__(self):
        self.llm = Ollama(model="gemma2")
        self.model = SentenceTransformer('fine-tuned/jinaai_jina-embeddings-v2-base-es-2572024-mb4o-webapp', trust_remote_code=True)


        # Cargar respuestas y generar embeddings
        df = pd.read_csv('Base_conocimiento_pre.csv', delimiter=';')
        
        # Filtrar respuestas que no sean NaN y convertir todas las respuestas a cadenas
        self.responses = df['Respuesta'].dropna().astype(str).tolist()
        
        # Verifica que todas las respuestas sean cadenas de texto
        print(f"Respuestas cargadas: {self.responses}")

        # Generar embeddings de las respuestas
        self.response_embeddings = self.model.encode(self.responses)

        # Crear el vectorstore de Chroma
        self.vectorstore = Chroma(embedding_function=SentenceTransformerEmbeddings(model_name=self.model), collection=self.collection)
        
        # Configuración del modelo Llama3 con gemma2 usando Ollama
        

    def truncate_vector(self, vector, size=None):
        # Truncar vector para mantener la dimensión esperada
        size = size or self.vector_dimension
        return vector[:size] if len(vector) > size else vector

    def get_best_response_from_second_bot(self, query, threshold=0.5):
        # Obtener la mejor respuesta del segundo bot (utilizando el modelo de embeddings)
        query_embedding = self.model.encode([query])[0]
        similarities = util.pytorch_cos_sim(query_embedding, self.response_embeddings)[0]
        max_similarity = similarities.max().item()
        best_response_idx = similarities.argmax().item()
        if max_similarity < threshold:
            return None
        return self.responses[best_response_idx]

    def get_response_from_first_bot(self, user_message):
        # Buscar la respuesta en Chroma
        user_embedding = self.model.encode([user_message])[0]
        truncated_embedding = self.truncate_vector(user_embedding)
        results = self.vectorstore.similarity_search(query_texts=[user_message])
    
        print(f"Resultados obtenidos: {results}")  # Depuración
    
        # Asegurarse de que results sea una lista de diccionarios
        if isinstance(results, list) and len(results) > 0 and isinstance(results[0], dict) and 'answer' in results[0]:
            return results[0]['answer']
    
        # Si no hay resultados válidos, generar respuesta con gemma2 usando Ollama
        response = self.llm.generate_response(user_message)
    
        # Agregar la nueva respuesta a la base de datos de Chroma
        self.vectorstore.add_texts(texts=[user_message], metadatas=[{'answer': response}])
    
        return response



# Uso del chatbot con gemma2 a través de Ollama
chatbot = ChatbotModel()
respuesta = chatbot.get_response_from_first_bot("¿Dónde se encuentra la biblioteca principal?")
print(respuesta)


Respuestas cargadas: ['En el bloque H\xa0', 'En bienestar estudiantil, junto al bar\xa0', 'Una vez al año', 'Dos accesos  para vehiculos y peatones', 'ASOCIACIÓN DE INGENIEROS MECÁNICOS “ASME”\r\nCHACAREROS IASA\r\nROBÓTICA\r\nRAMA ESTUDIANTIL IEEE\r\nPOLÍTICA EXTERIOR Y DIPLOMACIA\r\nGRUPO ESTUDIANTIL (Earthquake Engineering Research Institute) “EERI-ESPE”\r\nECOLÓGICO\r\nDESARROLLO DE SOFTWARE\r\nFOTOGRAFÍA\r\nDANZA\r\nMÚSICA (Ensamble)\r\nORATORIA\r\nLIDERAZGO', 'Según el articulo 5 de la LOES, los derechos de los estudiantes son:\r\n\r\na) Acceder, movilizarse, permanecer, egresar y titularse sin discriminación conforme sus méritos académicos.\r\n\r\nb) Acceder a una educación superior de calidad y pertinente, que permita iniciar una carrera académica y/o profesional en igualdad de oportunidades.\r\n\r\nc) Contar y acceder a los medios y recursos adecuados para su formación superior; garantizados por la Constitución.\r\n\r\nd) Participar en el proceso de evaluación y acreditación d

C:\Users\Liak\AppData\Roaming\Python\Python311\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 0.3.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEmbeddings`.
  warn_deprecated(


ValidationError: 1 validation error for HuggingFaceEmbeddings
model_name
  str type expected (type=type_error.str)

In [8]:
import pandas as pd
from sentence_transformers import SentenceTransformer
import chromadb
from chromadb.config import Settings
from chromadb import Client
from langchain_community.llms.ollama import Ollama
from langchain_core.messages import HumanMessage, AIMessage
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

# Cargar el archivo CSV con delimitador ;
file_path = 'Base_conocimiento_pre.csv'  # Actualiza el path si es necesario
df = pd.read_csv(file_path, delimiter=';')
# Limpiar datos faltantes en la columna 'Pregunta'
df['Pregunta'] = df['Pregunta'].fillna('')

# Cargar el modelo de embeddings
model = SentenceTransformer('all-MiniLM-L6-v2')

# Crear embeddings para las preguntas
df['question_embeddings'] = df['Pregunta'].apply(lambda x: model.encode(x).tolist())

# Configurar Chroma
settings = Settings()
client = Client(settings=settings)

# Crear una colección
collection = client.create_collection(name="qa_collection")

# Agregar vectores a la colección
for index, row in df.iterrows():
    collection.add(
        documents=[row['Pregunta']],
        embeddings=[row['question_embeddings']],
        ids=[str(index)]  # Cambiado metadatas a ids
    )

def obtener_respuesta(pregunta: str, collection, df, model):
     # Codificar la pregunta
    query_embedding = model.encode(pregunta).tolist()

    # Realizar la consulta en la colección
    results = collection.query(query_embeddings=[query_embedding], n_results=1)

    # Obtener las listas de IDs y distancias
    ids_list = results['ids'][0]
    distances_list = results['distances'][0]

    if not ids_list:  # No hay resultados
        return None, float('inf')

    # Combina los IDs y distancias en un solo objeto
    results_combined = list(zip(ids_list, distances_list))

    # Ordena los resultados por distancia (menor distancia significa mayor similitud)
    sorted_results = sorted(results_combined, key=lambda x: x[1])

    # Selecciona solo el mejor resultado (menor distancia)
    best_result = sorted_results[0]  # Solo el primer resultado es el mejor

    doc_id, distance = best_result
    index = int(doc_id)

    # Obtener la respuesta desde el DataFrame
    respuesta = df.loc[index, 'Respuesta']

    return respuesta, distance

# Configuración del modelo Llama3
llm = Ollama(model="gemma2")

prompt_template = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            """Eres una IA llamada Espesito. Tu tarea principal es responder preguntas simples sobre la Universidad de las Fuerzas Armadas "ESPE", basándote en la información proporcionada en las preguntas frecuentes.
            Cuando no tengas una respuesta exacta, responde de acuerdo a lo que has aprendido y el contexto dado en el historial de chat.""",
        ),
        MessagesPlaceholder(variable_name="chat_history"),
        ("human", "{input}"),
    ]
)

chain = prompt_template | llm

def chat():
    chat_history = []
    umbral_similitud = 0.5  # Ajusta este umbral según sea necesario
    while True:
        pregunta = input("You: ")
        if pregunta.lower() == "adios":
            return
        
        # Primero, intenta obtener una respuesta de Chroma
        answer, distance = obtener_respuesta(pregunta, collection, df, model)
        
        if answer and distance < umbral_similitud:
            # Usa la respuesta recuperada para generar una respuesta más elaborada con Gemma2
            context = f"La respuesta a tu pregunta es: {answer}"
            response = chain.invoke(input=context, chat_history=chat_history)
        else:
            # Si no se encuentra una respuesta adecuada, usa el modelo Llama3 directamente
            response = chain.run(input=pregunta, chat_history=chat_history)
        
        print(f"Espesito: {response}")
        chat_history.append(HumanMessage(content=pregunta))
        chat_history.append(AIMessage(content=response))

# Ejecutar el chat
chat()


UniqueConstraintError: Collection qa_collection already exists

In [1]:
import pandas as pd
from sentence_transformers import SentenceTransformer
import chromadb
from chromadb.config import Settings
from chromadb import Client
from langchain_community.llms.ollama import Ollama
from langchain_core.messages import HumanMessage, AIMessage
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

# Cargar el archivo CSV con delimitador ;
file_path = 'Base_conocimiento_pre.csv'  # Actualiza el path si es necesario
df = pd.read_csv(file_path, delimiter=';')
# Limpiar datos faltantes en la columna 'Pregunta'
df['Pregunta'] = df['Pregunta'].fillna('')

# Cargar el modelo de embeddings
model = SentenceTransformer('all-MiniLM-L6-v2')

# Crear embeddings para las preguntas
df['question_embeddings'] = df['Pregunta'].apply(lambda x: model.encode(x).tolist())

# Configurar Chroma
settings = Settings()
client = Client(settings=settings)

# Crear una colección
collection = client.create_collection(name="qa_collection")

# Agregar vectores a la colección
for index, row in df.iterrows():
    collection.add(
        documents=[row['Pregunta']],
        embeddings=[row['question_embeddings']],
        ids=[str(index)]  # Cambiado metadatas a ids
    )

def obtener_respuesta(pregunta: str, collection, df, model):
        # Codificar la pregunta
    query_embedding = model.encode(pregunta).tolist()

    # Realizar la consulta en la colección
    results = collection.query(query_embeddings=[query_embedding], n_results=1)

    # Obtener las listas de IDs y distancias
    ids_list = results['ids'][0]
    distances_list = results['distances'][0]

    if not ids_list:  # No hay resultados
        return None, float('inf')

    # Combina los IDs y distancias en un solo objeto
    results_combined = list(zip(ids_list, distances_list))

    # Ordena los resultados por distancia (menor distancia significa mayor similitud)
    sorted_results = sorted(results_combined, key=lambda x: x[1])

    # Selecciona solo el mejor resultado (menor distancia)
    best_result = sorted_results[0]  # Solo el primer resultado es el mejor

    doc_id, distance = best_result
    index = int(doc_id)

    # Obtener la respuesta desde el DataFrame
    respuesta = df.loc[index, 'Respuesta']

    return respuesta, distance

# Configuración del modelo Llama3
llm = Ollama(model="gemma2")

prompt_template = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            """Eres una IA llamada Espesito. Tu tarea principal es responder preguntas simples sobre la Universidad de las Fuerzas Armadas "ESPE", basándote en la información proporcionada en las preguntas frecuentes.
            Cuando no tengas una respuesta exacta, responde de acuerdo a lo que has aprendido y el contexto dado en el historial de chat.""",
        ),
        MessagesPlaceholder(variable_name="chat_history"),
        ("human", "{input}"),
    ]
)

chain = prompt_template | llm

def chat():
    chat_history = []
    umbral_similitud = 0.5  # Ajusta este umbral según sea necesario
    while True:
        pregunta = input("You: ")
        if pregunta.lower() == "adios":
            return
        
        # Primero, intenta obtener una respuesta de Chroma
        answer, distance = obtener_respuesta(pregunta, collection, df, model)
        
        if answer and distance < umbral_similitud:
            # Usa la respuesta recuperada para generar una respuesta más elaborada con Gemma2
            context = f"La respuesta a tu pregunta es: {answer}"
            chat_history.append(HumanMessage(content=pregunta))
            chat_history.append(AIMessage(content=answer))
        else:
            # Si no hay una respuesta adecuada en Chroma, usa el modelo Llama3
            response = chain.invoke(input=pregunta, chat_history=chat_history)
            chat_history.append(HumanMessage(content=pregunta))
            chat_history.append(AIMessage(content=response))
            print(f"Espesito: {response}")

# Iniciar el chat
chat()


C:\Users\Liak\AppData\Roaming\Python\Python311\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


You:  hola!


TypeError: BasePromptTemplate.invoke() got an unexpected keyword argument 'chat_history'

In [4]:
def chat():
    chat_history = []
    umbral_similitud = 0.5  # Ajusta este umbral según sea necesario
    while True:
        pregunta = input("You: ")
        if pregunta.lower() == "adios":
            return
        
        # Primero, intenta obtener una respuesta de Chroma
        answer, distance = obtener_respuesta(pregunta, collection, df, model)
        
        if answer and distance < umbral_similitud:
            # Usa la respuesta recuperada para generar una respuesta más elaborada con Gemma2
            context = f"La respuesta a tu pregunta es: {answer}"
            chat_history.append(HumanMessage(content=pregunta))
            chat_history.append(AIMessage(content=answer))
            print(f"Espesito: {answer}")
        else:
            # Si no hay una respuesta adecuada en Chroma, usa el modelo Llama3
            response = chain.invoke({"input": pregunta, "chat_history": chat_history})

            chat_history.append(HumanMessage(content=pregunta))
            chat_history.append(AIMessage(content=response))
            print(f"Espesito: {response}")

# Iniciar el chat


In [6]:
chat()


You:  Hola


Espesito: ¡Hola! 👋  Soy Espesito, tu asistente virtual para preguntas sobre la Universidad de las Fuerzas Armadas "ESPE". ¿Qué te gustaría saber? 😊  



You:  ¿Quien es coordinador de pasantías del DCCO?


Espesito: Phd. Edison Jorge Lascano 


You:  y sabes quien es el director de carrera de software?


...

You:  Y sabes cada cuanto tiempo se aplican estas becas?


Espesito: Una vez al año


You:  Y donde puedo ir a hablar de esto?


Espesito: Puedes dirigirte a la oficina de Becas de la ESPE.  

Te recomiendo que visites su página web oficial para encontrar información sobre sus horarios de atención y cómo contactarlos. 😊 



You:  adios


In [1]:
import pandas as pd
from sentence_transformers import SentenceTransformer
import chromadb
from chromadb.config import Settings
from chromadb import Client
from langchain_community.llms.ollama import Ollama
from langchain_core.messages import HumanMessage, AIMessage
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

# Cargar el archivo CSV con delimitador ;
file_path = 'Base_conocimiento_pre.csv'  # Actualiza el path si es necesario
df = pd.read_csv(file_path, delimiter=';')
# Limpiar datos faltantes en la columna 'Pregunta'
df['Pregunta'] = df['Pregunta'].fillna('')

# Cargar el modelo de embeddings
model = SentenceTransformer('all-MiniLM-L6-v2')

# Crear embeddings para las preguntas
df['question_embeddings'] = df['Pregunta'].apply(lambda x: model.encode(x).tolist())

# Configurar Chroma
settings = Settings()
client = Client(settings=settings)

# Crear una colección
collection = client.create_collection(name="qa_collection")

# Agregar todos los documentos a la colección de una sola vez
documents = df['Pregunta'].tolist()
embeddings = df['question_embeddings'].tolist()
ids = df.index.astype(str).tolist()

collection.add(
    documents=documents,
    embeddings=embeddings,
    ids=ids
)

# Hasta aquí, todas las preguntas y respuestas se han cargado en Chroma.

def obtener_respuesta(pregunta: str, collection, df, model):
    query_embedding = model.encode(pregunta).tolist()
    results = collection.query(query_embeddings=[query_embedding], n_results=1)
    ids_list = results['ids'][0]
    distances_list = results['distances'][0]

    if not ids_list:
        return None, float('inf')

    results_combined = list(zip(ids_list, distances_list))
    sorted_results = sorted(results_combined, key=lambda x: x[1])
    best_result = sorted_results[0]
    doc_id, distance = best_result
    index = int(doc_id)
    respuesta = df.loc[index, 'Respuesta']

    return respuesta, distance

# Configuración del modelo Llama3
llm = Ollama(model="gemma2")

prompt_template = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            """Eres una IA llamada Espesito. Tu tarea principal es responder preguntas simples sobre la Universidad de las Fuerzas Armadas "ESPE", basándote en la información proporcionada en las preguntas frecuentes.
            Cuando no tengas una respuesta exacta, responde de acuerdo a lo que has aprendido y el contexto dado en el historial de chat.""",
        ),
        MessagesPlaceholder(variable_name="chat_history"),
        ("human", "{input}"),
    ]
)

chain = prompt_template | llm

def chat():
    chat_history = []
    umbral_similitud = 0.5
    while True:
        pregunta = input("You: ")
        if pregunta.lower() == "adios":
            return
        
        answer, distance = obtener_respuesta(pregunta, collection, df, model)
        
        if answer and distance < umbral_similitud:
            context = f"La respuesta a tu pregunta es: {answer}"
            chat_history.append(HumanMessage(content=pregunta))
            chat_history.append(AIMessage(content=answer))
            print(f"Espesito: {answer}")
        else:
            response = chain.invoke({"input": pregunta, "chat_history": chat_history})
            chat_history.append(HumanMessage(content=pregunta))
            chat_history.append(AIMessage(content=response))
            print(f"Espesito: {response}")



C:\Users\Liak\AppData\Roaming\Python\Python311\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


In [2]:
chat()

You:  hola


Espesito: ¡Hola! Soy Espesito, tu asistente virtual para todo lo relacionado con la Universidad de las Fuerzas Armadas "ESPE". ¿Qué te gustaría saber? 😊  



You:  ueria hacer las practicas preprofesional o pasantias pero no se con quien hablar porque soy del departamento DCCO


Espesito: ¡Hola! Entiendo que quieres realizar prácticas preprofesionales o pasantías desde el departamento de DCCO. 

Para darte información más precisa, ¿podrías decirme qué tipo de práctica te interesa?  Por ejemplo, ¿buscas una práctica en desarrollo de software, análisis de datos, seguridad informática...?

Con más detalles podré orientarte mejor sobre a quién contactar en ESPE para tu práctica preprofesional. 😊



You:  pero quien es el coordinador


Espesito: ¡Claro que sí! Para prácticas preprofesionales del departamento DCCO, puedes contactarte con **el profesor [Nombre del Coordinador]**, quien es el responsable de coordinar estas experiencias. 


Para obtener su correo electrónico o número telefónico exacto, te recomiendo consultar la página web del Departamento de Ciencias y Computación o comunicarte con la oficina administrativa de DCCO. ¡Espero que esto te ayude! 😊



You:  adios


In [3]:
def obtener_respuesta(pregunta: str, collection, df, model):
    query_embedding = model.encode(pregunta).tolist()
    results = collection.query(query_embeddings=[query_embedding], n_results=3)  # Recuperar más resultados

    ids_list = results['ids'][0]
    distances_list = results['distances'][0]

    if not ids_list:
        return None, float('inf')

    results_combined = list(zip(ids_list, distances_list))
    sorted_results = sorted(results_combined, key=lambda x: x[1])
    best_result = sorted_results[0]
    doc_id, distance = best_result
    index = int(doc_id)
    respuesta = df.loc[index, 'Respuesta']

    # Recuperar respuestas adicionales para proporcionar contexto
    additional_context = [df.loc[int(doc_id), 'Respuesta'] for doc_id, _ in sorted_results[1:3]]

    return respuesta, distance, additional_context


In [4]:
def chat():
    chat_history = []
    umbral_similitud = 0.5
    while True:
        pregunta = input("You: ")
        if pregunta.lower() == "adios":
            return
        
        answer, distance, additional_context = obtener_respuesta(pregunta, collection, df, model)
        
        if answer and distance < umbral_similitud:
            context = f"La respuesta a tu pregunta es: {answer}. Información adicional: {' '.join(additional_context)}"
            chat_history.append(HumanMessage(content=pregunta))
            chat_history.append(AIMessage(content=answer))
            print(f"Espesito: {answer}")
        else:
            response = chain.invoke({"input": pregunta, "chat_history": chat_history})
            chat_history.append(HumanMessage(content=pregunta))
            chat_history.append(AIMessage(content=response))
            print(f"Espesito: {response}")


In [5]:
chat()

You:  Hola


Espesito: ¡Hola! Bienvenido a ESPE. ¿En qué puedo ayudarte hoy? 😄 



You:  queria hacer las pasantias pero no se con quien hablar porque soy del departamento DCCO y tampoco se el bloque al que debo ir


Espesito: ¡Entiendo tu situación!  Como estudiante del DCCO, para información sobre pasantías debes dirigirte a la oficina del **Departamento de Prácticas Universitarias**.  

¿Te gustaría que te ayude a encontrar la ubicación o los datos de contacto de esa oficina? 😊 



You:  quien es el coordinador


...

You:  Osea si se encuentra ahi el coordinador?


Espesito: Si, según la información que has proporcionado, el coordinador de pasantías del DCCO,  Phd. Edison Jorge Lascano, trabaja en el departamento de Computación ubicado en el bloque H.





You:  y el director tambiem?


Espesito: El director de tu carrera se llama Ing. Mauricio Camapaña Alias "Monster", pero no puedo decirte en qué bloque está. Te recomiendo que contactes con la oficina administrativa del DCCO o consultes el directorio de ESPE en su página web. 😊 





You:  adios


In [1]:
import pandas as pd
from sentence_transformers import SentenceTransformer
import chromadb
from chromadb.config import Settings
from chromadb import Client
from langchain_community.llms.ollama import Ollama
from langchain_core.messages import HumanMessage, AIMessage
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

# Cargar el archivo CSV con delimitador ;
file_path = 'Base_conocimiento_pre.csv'  # Actualiza el path si es necesario
df = pd.read_csv(file_path, delimiter=';')
df['Pregunta'] = df['Pregunta'].fillna('')

# Cargar el modelo de embeddings
model = SentenceTransformer('all-MiniLM-L6-v2')

# Crear embeddings para las preguntas
df['question_embeddings'] = df['Pregunta'].apply(lambda x: model.encode(x).tolist())

# Configurar Chroma
settings = Settings()
client = Client(settings=settings)

# Crear una colección
collection = client.create_collection(name="qa_collection")

# Agregar todos los documentos a la colección de una sola vez
documents = df['Pregunta'].tolist()
embeddings = df['question_embeddings'].tolist()
ids = df.index.astype(str).tolist()

collection.add(
    documents=documents,
    embeddings=embeddings,
    ids=ids
)

def obtener_respuesta(pregunta: str, collection, df, model):
    query_embedding = model.encode(pregunta).tolist()
    results = collection.query(query_embeddings=[query_embedding], n_results=3)

    ids_list = results['ids'][0]
    distances_list = results['distances'][0]

    if not ids_list:
        return None, float('inf')

    results_combined = list(zip(ids_list, distances_list))
    sorted_results = sorted(results_combined, key=lambda x: x[1])
    best_result = sorted_results[0]
    doc_id, distance = best_result
    index = int(doc_id)
    respuesta = df.loc[index, 'Respuesta']

    additional_context = [df.loc[int(doc_id), 'Respuesta'] for doc_id, _ in sorted_results[1:3]]

    return respuesta, distance, additional_context

# Configuración del modelo Ollama
llm = Ollama(model="gemma2")

prompt_template = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            """Eres una IA llamada Espesito. Tu tarea principal es responder preguntas simples sobre la Universidad de las Fuerzas Armadas "ESPE", basándote en la información proporcionada en las preguntas frecuentes.
            Cuando no tengas una respuesta exacta, responde de acuerdo a lo que has aprendido y el contexto dado en el historial de chat.""",
        ),
        MessagesPlaceholder(variable_name="chat_history"),
        ("human", "{input}"),
    ]
)

chain = prompt_template | llm

def chat():
    chat_history = []
    umbral_similitud = 0.5
    while True:
        pregunta = input("You: ")
        if pregunta.lower() == "adios":
            break
        
        answer, distance, additional_context = obtener_respuesta(pregunta, collection, df, model)
        
        if answer and distance < umbral_similitud:
            context = f"La respuesta a tu pregunta es: {answer}. Información adicional: {' '.join(additional_context)}"
            chat_history.append(HumanMessage(content=pregunta))
            chat_history.append(AIMessage(content=answer))
            print(f"Espesito: {answer}")
        else:
            response = chain.invoke({"input": pregunta, "chat_history": chat_history})
            chat_history.append(HumanMessage(content=pregunta))
            chat_history.append(AIMessage(content=response))
            print(f"Espesito: {response}")



C:\Users\Liak\AppData\Roaming\Python\Python311\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


In [2]:
chat()

You:  hola


Espesito: ¡Hola! 👋 Soy Espesito, tu asistente informativo sobre la Universidad de las Fuerzas Armadas "ESPE". ¿Tienes alguna pregunta para mí? 😊  



You:  ueria hacer las pasantias y como soy del departamento del DCCO queria saber quien es el coordinador 


Espesito: Phd. Edison Jorge Lascano 


You:  y sabes donde se encuentra


...

You:  y sabes en donde esta este bloque


Espesito: El Bloque de Ciencias Computación (DCCO) se encuentra en el campus norte de la ESPE.  

¿Necesitas más información sobre el campus o cómo llegar al DCCO? 🗺️ 



You:  Y cual es el bloque


Espesito: El bloque del DCCO se llama **Bloque C**.  



You:  adios


In [5]:
import pandas as pd
import torch  # Asegúrate de importar torch
from sentence_transformers import SentenceTransformer
import chromadb
from chromadb.config import Settings
from chromadb import Client
from langchain_community.llms.ollama import Ollama
from langchain_core.messages import HumanMessage, AIMessage
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Chroma
from langchain_core.documents import Document  # Importa la clase Document

# Cargar el archivo CSV con delimitador ;
file_path = 'Base_conocimiento_pre.csv'  # Actualiza el path si es necesario
df = pd.read_csv(file_path, delimiter=';')
df['Pregunta'] = df['Pregunta'].fillna('')

# Dividir los datos en fragmentos si es necesario
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)
documents = [Document(page_content=row['Pregunta']) for _, row in df.iterrows()]  # Crea objetos Document
all_splits = text_splitter.split_documents(documents)

# Crear embeddings para las preguntas
model_name = "sentence-transformers/all-mpnet-base-v2"
model_kwargs = {"device": "cuda" if torch.cuda.is_available() else "cpu"}

try:
    embeddings = HuggingFaceEmbeddings(model_name=model_name, model_kwargs=model_kwargs)
except Exception as ex:
    print("Exception: ", ex)
    local_model_path = "sentence-transformers/all-MiniLM-L6-v2"
    print(f"Use alternative (local) model: {local_model_path}\n")
    embeddings = HuggingFaceEmbeddings(model_name=local_model_path, model_kwargs=model_kwargs)

# Configurar Chroma
settings = Settings()
client = Client(settings=settings)

# Crear una colección
collection = Chroma.from_documents(documents=all_splits, embedding=embeddings, persist_directory="chroma_db")

def obtener_respuesta(pregunta: str, collection, df, model):
    query_embedding = model.encode(pregunta).tolist()
    results = collection.query(query_embeddings=[query_embedding], n_results=3)

    ids_list = results['ids'][0]
    distances_list = results['distances'][0]

    if not ids_list:
        return None, float('inf')

    results_combined = list(zip(ids_list, distances_list))
    sorted_results = sorted(results_combined, key=lambda x: x[1])
    best_result = sorted_results[0]
    doc_id, distance = best_result
    index = int(doc_id)
    respuesta = df.loc[index, 'Respuesta']

    additional_context = [df.loc[int(doc_id), 'Respuesta'] for doc_id, _ in sorted_results[1:3]]

    return respuesta, distance, additional_context

# Configuración del modelo Ollama
llm = Ollama(model="gemma2")

prompt_template = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            """Eres una IA llamada Espesito. Tu tarea principal es responder preguntas simples sobre la Universidad de las Fuerzas Armadas "ESPE", basándote en la información proporcionada en las preguntas frecuentes.
            Cuando no tengas una respuesta exacta, responde de acuerdo a lo que has aprendido y el contexto dado en el historial de chat.""",
        ),
        MessagesPlaceholder(variable_name="chat_history"),
        ("human", "{input}"),
    ]
)

chain = prompt_template | llm

def chat():
    chat_history = []
    umbral_similitud = 0.5
    while True:
        pregunta = input("You: ")
        if pregunta.lower() == "adios":
            break
        
        answer, distance, additional_context = obtener_respuesta(pregunta, collection, df, embeddings)  # Asegúrate de pasar 'embeddings' en lugar de 'model'
        
        if answer and distance < umbral_similitud:
            context = f"La respuesta a tu pregunta es: {answer}. Información adicional: {' '.join(additional_context)}"
            chat_history.append(HumanMessage(content=pregunta))
            chat_history.append(AIMessage(content=answer))
            print(f"Espesito: {answer}")
        else:
            response = chain.invoke({"input": pregunta, "chat_history": chat_history})
            chat_history.append(HumanMessage(content=pregunta))
            chat_history.append(AIMessage(content=response))
            print(f"Espesito: {response}")


C:\Users\Liak\AppData\Roaming\Python\Python311\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 0.3.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEmbeddings`.
  warn_deprecated(


In [1]:
import pandas as pd
import torch
from sentence_transformers import SentenceTransformer
import chromadb
from chromadb.config import Settings
from chromadb import Client
from langchain_community.llms.ollama import Ollama
from langchain_core.messages import HumanMessage, AIMessage
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Chroma
from langchain_core.documents import Document

# Cargar el archivo CSV con delimitador ;
file_path = 'Base_conocimiento_pre.csv'
df = pd.read_csv(file_path, delimiter=';')
df['Pregunta'] = df['Pregunta'].fillna('')

# Dividir los datos en fragmentos si es necesario
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)
documents = [Document(page_content=row['Pregunta']) for _, row in df.iterrows()]
all_splits = text_splitter.split_documents(documents)

# Crear embeddings para las preguntas
model_name = "sentence-transformers/all-mpnet-base-v2"
model_kwargs = {"device": "cuda" if torch.cuda.is_available() else "cpu"}

try:
    embeddings_model = HuggingFaceEmbeddings(model_name=model_name, model_kwargs=model_kwargs)
except Exception as ex:
    print("Exception: ", ex)
    local_model_path = "sentence-transformers/all-MiniLM-L6-v2"
    print(f"Use alternative (local) model: {local_model_path}\n")
    embeddings_model = HuggingFaceEmbeddings(model_name=local_model_path, model_kwargs=model_kwargs)

# Configurar Chroma
settings = Settings()
client = Client(settings=settings)
collection = client.create_collection(name="qa_collection")

# Generar embeddings para todos los fragmentos
all_embeddings = [embeddings_model.embed_documents([doc.page_content])[0] for doc in all_splits]
ids = [str(i) for i in range(len(all_splits))]

# Agregar todos los documentos a la colección
collection.add(
    documents=[doc.page_content for doc in all_splits],
    embeddings=all_embeddings,
    ids=ids
)

def obtener_respuesta(pregunta: str, collection, df, model):
    # Obtener el embedding de la pregunta
    query_embedding = model.embed_documents([pregunta])[0]

    # Realizar la consulta en la colección usando similarity_search
    results = collection.query(
        query_embeddings=[query_embedding],  # Consulta con embeddings
        n_results=3  # Número de resultados a devolver
    )

    ids_list = results['ids'][0]
    distances_list = results['distances'][0]

    if not ids_list:
        return None, float('inf'), []

    results_combined = list(zip(ids_list, distances_list))
    sorted_results = sorted(results_combined, key=lambda x: x[1])
    best_result = sorted_results[0]
    doc_id, distance = best_result
    index = int(doc_id)
    respuesta = df.loc[index, 'Respuesta']

    # Contexto adicional
    additional_context = [df.loc[int(doc_id), 'Respuesta'] for doc_id, _ in sorted_results[1:3]]

    return respuesta, distance, additional_context

# Configuración del modelo Ollama
llm = Ollama(model="gemma2")

prompt_template = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            """Eres una IA llamada Espesito. Tu tarea principal es responder preguntas simples sobre la Universidad de las Fuerzas Armadas "ESPE", basándote en la información proporcionada en las preguntas frecuentes.
            Cuando no tengas una respuesta exacta, responde de acuerdo a lo que has aprendido y el contexto dado en el historial de chat. Además siempre debes responder en español""",
        ),
        MessagesPlaceholder(variable_name="chat_history"),
        ("human", "{input}"),
    ]
)

chain = prompt_template | llm

def chat():
    chat_history = []
    umbral_similitud = 0.5
    while True:
        pregunta = input("You: ")
        if pregunta.lower() == "adios":
            break
        
        answer, distance, additional_context = obtener_respuesta(pregunta, collection, df, embeddings_model)
        
        if answer and distance < umbral_similitud:
            context = f"La respuesta a tu pregunta es: {answer}. Información adicional: {' '.join(additional_context)}"
            chat_history.append(HumanMessage(content=pregunta))
            chat_history.append(AIMessage(content=answer))
            print(f"Espesito: {answer}")
        else:
            response = chain.invoke({"input": pregunta, "chat_history": chat_history})
            chat_history.append(HumanMessage(content=pregunta))
            chat_history.append(AIMessage(content=response))
            print(f"Espesito: {response}")

# Ejecutar el chat
chat()


C:\Users\Liak\AppData\Roaming\Python\Python311\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


C:\Users\Liak\AppData\Roaming\Python\Python311\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 0.3.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEmbeddings`.
  warn_deprecated(


You:  hola


Espesito: ¡Hola! 👋 Soy Espesito, tu asistente virtual para todas tus dudas sobre la Universidad de las Fuerzas Armadas "ESPE". ¿Qué te gustaría saber? 😊  



You:  iba a realizar las pasantias y queria hablar con el coordinador, sabes como se llama porque pertencezco al departamento DCCO


...

You:  y sabes en que edificio esta el departamento del DCCO


Espesito: Phd. Edison Jorge Lascano 


You:  me refiero al bloque


Espesito: El Departamento de Ciencia e Ingeniería Computacional (DCCO) se encuentra en el **edificio A**.  


Let me know if you have any other questions! 😊 



You:  adios


In [2]:
chat()


You:  adios
